https://keras.io/examples/nlp/text_classification_from_scratch/

In [1]:
import pandas as pd
from collections import Counter
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# TF tensors

In [2]:
# tf.constant creates a tensor
tf_tensor = tf.constant(["Привет", "Как дела"])

In [3]:
tf_tensor.shape

TensorShape([2])

In [4]:
# NB: Russian lang is represented as a byte strings -> that won't cause any problems
tf_tensor

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'\xd0\x9f\xd1\x80\xd0\xb8\xd0\xb2\xd0\xb5\xd1\x82',
       b'\xd0\x9a\xd0\xb0\xd0\xba \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0'],
      dtype=object)>

# Load csv data 

In [5]:
ls ../data/raw

lenta_10k_3_classes.csv  readme.md


In [6]:
path = '../data/raw/lenta_10k_3_classes.csv'

In [7]:
df = pd.read_csv(path, encoding = 'utf8')

In [54]:
# shuffle rows 
df = df.sample(frac = 1)

In [8]:
# df = df[:1000]

In [9]:
df.head()

,text,topic
0,С 1 января 2000 года все телеканалы будут опла...,Экономика
1,Германский автопромышленный концерн Volkswagen...,Экономика
2,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика
3,Две крупнейших телекоммуникационных компании С...,Экономика
4,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика


In [10]:
# Take a glance at unique classes
df['topic'].unique()

array(['Экономика', 'Спорт', 'Культура'], dtype=object)

In [11]:
# Distributions
Counter(df['topic'])

Counter({'Экономика': 4902, 'Спорт': 2170, 'Культура': 2928})

In [12]:
def create_dir_path(*dir_names):
    """
    Concatenates variable num of dir names using forward slash
    """
    
    path = ''
    for name in dir_names:
        path += "/" + name
    
    return path[1:]

In [13]:
def create_text_files_dir(df, text_col='text', label_col='topic', dir_root_name='dataset', path='../data/interim'):
    """
    Creates a text dir w/ the following structure from a dataframe:
     main_directory/  
    ...class_a/  
    ......a_text_1.txt  
    ......a_text_2.txt  
    ...class_b/  
    ......b_text_1.txt  
    ......b_text_2.txt
    """
    
    # create dirs with label names if they don't exhist yet
    class_names = df['topic'].unique()
    for class_name in class_names:
        class_name_dir_path = create_dir_path(path, dir_root_name, class_name)
        
        if not os.path.exists(class_name_dir_path):
            os.makedirs(class_name_dir_path)
        
        # fill the dirs w/ text files
        class_df = df[df['topic']==class_name]
        for i, row in class_df.iterrows():
            with open(f'{class_name_dir_path}/{class_name}_text_{i}.txt', 'w+') as f:
                f.write(row['text'])

In [14]:
# create train and test dfs
train_df, test_df = train_test_split(df, test_size=0.2)

In [15]:
train_df.shape

(8000, 2)

In [16]:
test_df.shape

(2000, 2)

In [17]:
# creating directories
create_text_files_dir(train_df, dir_root_name='train')
create_text_files_dir(test_df, dir_root_name='test')

In [18]:
label_names = list(os.walk('../data/interim/test'))[0][1]
label_names

['Культура', 'Экономика', 'Спорт']

# Create train, val, test tf Datasets

In [19]:
# NB subset param, it enables taking diff splits -> seed must be the same
# label_mode='categorical' for categorical_crossentropy loss

In [20]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/train",
    label_mode='int',
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
    class_names=label_names
)

Found 8000 files belonging to 3 classes.
Using 6400 files for training.


In [21]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/train",
    label_mode='int',
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    class_names=label_names
    
)

Found 8000 files belonging to 3 classes.
Using 1600 files for validation.


In [22]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "../data/interim/test",
    batch_size=batch_size, 
    label_mode='int',
    class_names=label_names
)

Found 2000 files belonging to 3 classes.


In [23]:
print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Number of batches in raw_train_ds: 200
Number of batches in raw_val_ds: 50
Number of batches in raw_test_ds: 63


In [24]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(6):
        print(label_batch.numpy()[i])

0
0
0
0
1
2


# Vectorization

In [25]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [26]:
max_features = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [27]:
vocab = vectorize_layer.get_vocabulary()

In [28]:
len(vocab)

0

In [29]:
'акций' in vocab

False

In [30]:
vocab

[]

In [31]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [32]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [33]:
vectorize_text('Привет как дела', 1)

(<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
 array([[  1,  23, 846,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0]])>,
 1)

In [34]:
vectorize_layer.get_vocabulary()[707]

'АО'

In [35]:
# Apply verctorization to dataset like a function

In [36]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [37]:
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

# Model

In [38]:
embedding_dim = 300

In [39]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.LSTM(64)(x)
predictions = layers.Dense(3, activation="sigmoid", name="predictions")(x)

In [40]:
model = tf.keras.Model(inputs, predictions)
# opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
opt = 'Adam'

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [41]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
200/200 [==============================] - 14s 71ms/step - loss: 0.5921 - accuracy: 0.7742 - val_loss: 0.2848 - val_accuracy: 0.9187
Epoch 2/10
200/200 [==============================] - 13s 67ms/step - loss: 0.4011 - accuracy: 0.8175 - val_loss: 0.3801 - val_accuracy: 0.8263
Epoch 3/10
200/200 [==============================] - 14s 68ms/step - loss: 0.2676 - accuracy: 0.8741 - val_loss: 0.2594 - val_accuracy: 0.9062
Epoch 4/10
200/200 [==============================] - 14s 68ms/step - loss: 0.2447 - accuracy: 0.9280 - val_loss: 0.3020 - val_accuracy: 0.9144
Epoch 5/10
200/200 [==============================] - 14s 69ms/step - loss: 0.1828 - accuracy: 0.9509 - val_loss: 0.3524 - val_accuracy: 0.9013
Epoch 6/10
200/200 [==============================] - 14s 69ms/step - loss: 0.3798 - accuracy: 0.8730 - val_loss: 0.5340 - val_accuracy: 0.8012
Epoch 7/10
200/200 [==============================] - 14s 72ms/step - loss: 0.3936 - accuracy: 0.8458 - val_loss: 0.4971 - val_accuracy:

In [42]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

63/63 [==============================] - 1s 13ms/step - loss: 0.1627 - accuracy: 0.9235
Loss:  0.1626977026462555
Accuracy:  0.9235000014305115


# End to end model

In [43]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

63/63 [==============================] - 1s 15ms/step - loss: 0.1627 - accuracy: 0.9235


[0.1626976877450943, 0.9235000014305115]

In [44]:
import numpy as np

In [45]:
def predict(text):
    pred_probas = end_to_end_model.predict([text])
    idx = np.argmax(pred_probas)
    
    return label_names[idx]

# Sanity check

In [58]:
for i, row in df[:20].iterrows():
    print('Text: ', row['text'])
    print('True: ', row['topic'])
    print('Pred: ', predict(row['text']))
    print()

Text:  Министерство печати второй раз за два месяца выносит предупреждение с угрозой отзыва лицензии издательству за публикацию произведения, содержащего "непристойные сцены, провоцирующие низменные инстинкты". Об этом сообщает "РИА "Новости". Речь идет о тверском издательстве KOLONNA, выпустившем совместно с "Митиным" журналом  роман-трилогию Ильи Масодова "Мрак твоих глаз". Главные герои этого романа, написанного живущим в Германии 35-летним автором нарочито тяжеловесным "мертвым" языком, - убитые пионерки-упыри. Рецензенты, пишущие о романе, оперируют такими определениями, как "претендент на звание "самая стремная книга на русском языке", "хорошая садомазохистская порнография" и даже "источник чистой и нежной радости некро-педо-зоофила". Кроме того, в заключении экспертизы Минпечати особо отмечается, что "текст содержит вымыслы о героях Гражданской и Великой Отечественной войн, которым приписываются акты насилия и жестокости". Характерно, что, как и в случае с нашумевшим "Низшим пил

Pred:  Культура

Text:  Чеченская нефтяная компания будет создана до конца августа, она "безусловно  будет государственной". Об этом заявил журналистам в среду вице-премьер российского правительства Виктор Христенко. По его словам, в настоящее время с администрацией Чеченской Республики идет согласование вопроса о форме собственности компании. Обсуждаются, в частности, варианты унитарного предприятия, акционерного общества, а также других возможных форм собственности. Вице-премьер также сказал, что компания будет заниматься восстановлением нефтегазового комплекса Чечни и текущей добычей нефти, передает РИА "Новости".
True:  Экономика
Pred:  Экономика

Text:  В Министерстве путей сообщения готовится проект постановления об увеличении зонных тарифов на пригородные пассажирские перевозки, сообщает Федеральное агентство новостей. Этот вопрос обсуждался в субботу на расширенном заседании коллегии МПС. При этом подчеркивалось, что величина тарифов должна обеспечивать конкурентоспособность же

Pred:  Экономика

Text:  Музыка в стиле "хаус" способствует временному возникновению импотенции, заявили итальянские исследователи. Исследования, проведенные в Риме сотрудниками ассоциации Help Me среди 500 человек, показали, что 66% молодых людей в возрасте от 16 до 24 лет после прослушивания музыки "хаус" испытывают сексуальные проблемы, сообщает ananova.com. По мнению психологов, сильно выраженный ритм и почти полное отсутствие мелодии гасят сексуальные желания.
True:  Культура
Pred:  Культура



In [60]:
predict('На счета Минфина сегодня поступили средства японского государственного')

'Экономика'

In [62]:
predict(' понедельник стал известен состав лыжной сборной России, которая выступит на чемпионате мира в финском Лахти')

'Спорт'